In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import tensorflow.contrib.learn as tflearn
import tensorflow.contrib.layers as tflayers
import tensorflow.contrib.metrics as metrics
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, roc_auc_score, accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SVMSMOTE
from imblearn.under_sampling import NearMiss
from scipy.stats import norm

tf.set_random_seed(1)
np.random.seed(1)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.


In [2]:
df = pd.read_csv('creditcard.csv')
df = df.sample(frac=1).reset_index(drop=True)

In [3]:
#Original Data => Original test & Original Train using Striatified for saving data structure
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold, StratifiedKFold

x = df.drop(['Class','Time'], axis=1)
y = df['Class']

skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in skf.split(x, y):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

x_train = tf.keras.utils.normalize(x_train.values, axis = 1, order = 1) 
x_test = tf.keras.utils.normalize(x_test.values, axis = 1, order = 1) 
y_train = y_train.values
y_test = y_test.values

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
np.sum(y_train)

(227846, 29) (227846, 1) (56961, 29) (56961, 1)


394

In [4]:
# NN model
from sklearn.metrics import mean_squared_error

epochs = 2000
batch_size = 100
lr = 1e-3
hidden_layers1 = 20
hidden_layers2 = 15

# placeholders
X = tf.placeholder(tf.float32, (None,29), name="X")
Y = tf.placeholder(tf.float32, (None,1), name="Y")

# initializers
init_b = tf.keras.initializers.constant(0.0)
init_w = tf.keras.initializers.VarianceScaling(mode='fan_avg')

# layer 1
b1 = tf.get_variable('b1',(15,), tf.float32, init_b)
w1 = tf.get_variable('w1',(29,15), tf.float32, init_w)

# layer 2
b2 = tf.get_variable('b2',(10,), tf.float32, init_b)
w2 = tf.get_variable('w2',(15,10), tf.float32, init_w)

# activation
X1 = tf.nn.leaky_relu(X @ w1 + b1)
logits = X1 @ w2 + b2

y_pred = tf.cast(tf.argmax(logits, axis=1), tf.int32, name='y_pred_cls')
entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=logits)
loss = tf.reduce_mean(entropy)

# optimizer
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [5]:
init = tf.global_variables_initializer()

In [6]:
def shuffle_batch(x, y, batch_size):
    rnd_idx = np.random.permutation(len(x))
    n_batches = len(x) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        yield x[batch_idx], y[batch_idx]

In [7]:
with tf.Session() as sess:
    init.run()
    #saver.restore(sess, 'save/nn.ckpt')
    
    loss_trace = []
    accuracy_trace = []

    for i in range(epochs):
        print()
        print('epoch {}/{}'.format(i, epochs), end=' ')
        for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size):
            feed_dict = {x: x_batch, y: y_batch}
            loss_run, accuracy_run, _ = sess.run([loss, accuracy, train], feed_dict=feed_dict)
            loss_trace.append(loss_run)
            accuracy_trace.append(accuracy_run)

    feed_dict = {x: x_test, y: y_test}
    accuracy_run, y_test_pred = sess.run([accuracy, y_pred], feed_dict=feed_dict)


epoch 0/2000 

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed